In [14]:
# load packages
import pandas as pd
import geopandas as gpd
import unidecode 

In [15]:
pois_file_name = './gam_raw.json'
raw_df = pd.read_json(pois_file_name)
raw_df.count()

id               179
title            179
description      179
street           179
city             179
state            179
postal_code      179
lat              179
lng              179
phone            179
fax              179
email            179
website          179
logo_id            0
path               0
marker_id        179
description_2    179
open_hours       179
ordr             179
brand              0
categories       171
days_str         179
dtype: int64

In [16]:
pois_df = raw_df[['id', 'title', 'street', 'city', 'state', 'phone', 'fax','categories', 'lat', 'lng']]
pois_df.head()

,id,title,street,city,state,phone,fax,categories,lat,lng
0,1616,05-01.10 - Agence BOUABSA,08 avenue les frères BOUABSA-BATNA,Batna,BATNA,07 82 29 14 75 / 05 54 80 21 39,,108.0,35.554867,6.176896
1,1591,21.2.Y.BOULECHFAR,Route National El Hrouche SKIKDA,Sidi Mezghiche,CONSTANTINE,07 70 99 41 07,,105.0,36.655703,6.840460
2,1534,44-04.10 - Agence KHEMIS MELIANA,Cité 56 logts LSP khmis miliana,Khemis Miliana,AIN DEFLA,07 82 29 14 37 / 05 54 80 30 71,,94.0,36.256575,2.220505
3,1506,DZ001 - Agence CHERAGA,Chéraga Ouled Fayet N° 57 Alger,Chéraga,ALGER,07 82 29 15 86,,91.0,36.755765,2.951391
4,1525,02-01.50,RDC Batiment 4 N°1 commune et daira de Chlef,Chlef,CHLEF,0782 16 44 39 / 0554 84 86 74,,98.0,36.164586,1.331892


In [17]:
pois_df.rename(columns={'state': 'wilaya', 'lng': 'lon'}, inplace=True) 
pois_df.wilaya

/home/ali/.virtualenvs/geoweba/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


0            BATNA
1      CONSTANTINE
2        AIN DEFLA
3            ALGER
4            CHLEF
          ...     
174          ALGER
175          ALGER
176           ORAN
177           ORAN
178          SETIF
Name: wilaya, Length: 179, dtype: object

In [18]:
pois_df['wilaya_maj'] = [unidecode.unidecode(w) for w in pois_df.wilaya.str.upper()]
pois_df.wilaya_maj

<ipython-input-18-6b978fc75b42>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pois_df['wilaya_maj'] = [unidecode.unidecode(w) for w in pois_df.wilaya.str.upper()]


0            BATNA
1      CONSTANTINE
2        AIN DEFLA
3            ALGER
4            CHLEF
          ...     
174          ALGER
175          ALGER
176           ORAN
177           ORAN
178          SETIF
Name: wilaya_maj, Length: 179, dtype: object

In [19]:
# Add Wilayas codes
# Load Wilayas Features and calculate polygon centroids
wilayas_gjson = '../../../dz-admin/wilayas_48.csv'
wilayas_df = pd.read_csv(wilayas_gjson)
wilayas_df = wilayas_df[['code', 'nom_maj']]
wilayas_df = wilayas_df.rename(columns={'nom_maj': 'nom_wil', 'code': 'code_wil'})
wilayas_df.head()

,code_wil,nom_wil
0,1,ADRAR
1,2,CHLEF
2,3,LAGHOUAT
3,4,OUM EL BOUAGHI
4,5,BATNA


In [20]:
# Merge with Wialyas DataFrame
merged = pois_df.merge(wilayas_df, left_on='wilaya_maj', right_on='nom_wil', how='left')

merged[merged['code_wil'].isnull()]

,id,title,street,city,wilaya,phone,fax,categories,lat,lon,wilaya_maj,code_wil,nom_wil


In [21]:
merged.count()

id            179
title         179
street        179
city          179
wilaya        179
phone         179
fax           179
categories    171
lat           179
lon           179
wilaya_maj    179
code_wil      179
nom_wil       179
dtype: int64

In [22]:
merged.head()

,id,title,street,city,wilaya,phone,fax,categories,lat,lon,wilaya_maj,code_wil,nom_wil
0,1616,05-01.10 - Agence BOUABSA,08 avenue les frères BOUABSA-BATNA,Batna,BATNA,07 82 29 14 75 / 05 54 80 21 39,,108.0,35.554867,6.176896,BATNA,5,BATNA
1,1591,21.2.Y.BOULECHFAR,Route National El Hrouche SKIKDA,Sidi Mezghiche,CONSTANTINE,07 70 99 41 07,,105.0,36.655703,6.840460,CONSTANTINE,25,CONSTANTINE
2,1534,44-04.10 - Agence KHEMIS MELIANA,Cité 56 logts LSP khmis miliana,Khemis Miliana,AIN DEFLA,07 82 29 14 37 / 05 54 80 30 71,,94.0,36.256575,2.220505,AIN DEFLA,44,AIN DEFLA
3,1506,DZ001 - Agence CHERAGA,Chéraga Ouled Fayet N° 57 Alger,Chéraga,ALGER,07 82 29 15 86,,91.0,36.755765,2.951391,ALGER,16,ALGER
4,1525,02-01.50,RDC Batiment 4 N°1 commune et daira de Chlef,Chlef,CHLEF,0782 16 44 39 / 0554 84 86 74,,98.0,36.164586,1.331892,CHLEF,2,CHLEF


In [23]:
final_df = merged.drop(columns=['wilaya_maj', "wilaya", 'nom_wil'], axis=1)
final_df.rename_axis('num', inplace=True)
final_df.head()

,id,title,street,city,phone,fax,categories,lat,lon,code_wil
num,,,,,,,,,,
0,1616,05-01.10 - Agence BOUABSA,08 avenue les frères BOUABSA-BATNA,Batna,07 82 29 14 75 / 05 54 80 21 39,,108.0,35.554867,6.176896,5
1,1591,21.2.Y.BOULECHFAR,Route National El Hrouche SKIKDA,Sidi Mezghiche,07 70 99 41 07,,105.0,36.655703,6.840460,25
2,1534,44-04.10 - Agence KHEMIS MELIANA,Cité 56 logts LSP khmis miliana,Khemis Miliana,07 82 29 14 37 / 05 54 80 30 71,,94.0,36.256575,2.220505,44
3,1506,DZ001 - Agence CHERAGA,Chéraga Ouled Fayet N° 57 Alger,Chéraga,07 82 29 15 86,,91.0,36.755765,2.951391,16
4,1525,02-01.50,RDC Batiment 4 N°1 commune et daira de Chlef,Chlef,0782 16 44 39 / 0554 84 86 74,,98.0,36.164586,1.331892,2


In [24]:
final_df.to_csv('./gam_branches_to_check.csv', header = True)